# Intro

In [1]:
from pysmt.shortcuts import Not, And, Or, Symbol, TRUE, FALSE
from pysmt.shortcuts import GE, LE, LT, GT, Equals, Plus, Times, Real
from pysmt.shortcuts import Solver, Real
from pysmt.typing import REAL, INT

import numpy as np

In [2]:
from sage.all import Polyhedron

In [3]:
x = Symbol("x",REAL)
y = Symbol("y",REAL)
z = Symbol("z",REAL)


# formula = Not(And(Not(LE(x,Real(3))), GE(y,Real(1)), LE(y,Real(2)), Or(GE(y,Real(3)), LE(y,Real(2)))))
formula = Not(And(Not(LE(x,Real(3))), Not(LT(y,Real(1))), LE(y,Real(2)), Or(Not(LT(y,Real(3))), LT(y,Real(2)))))
formula.serialize()

'(! ((! (x <= 3.0)) & (! (y < 1.0)) & (y <= 2.0) & ((! (y < 3.0)) | (y < 2.0))))'

# Conversions

In [4]:
def inv(l):
    '''
    Invert each element of a list

    :param l: a list
    :return: a list
    '''

    return [-i for i in list(l)]


def H_to_array(P):
    '''
    Convert a polyhedron  P into two arrays A,b
    such that P = {x | Ax >= b}

    :param P: Polyhedron()
    :return: (A=np.array,b=np.array)
    '''

    H =  P.Hrepresentation()
    A = []
    b = []
    for ieq in H:
        if ieq.is_inequality():
            A.append(list(ieq.A()))
            b.append(ieq.b())
        elif ieq.is_equation():
            inv(list(ieq.A()))
            A.append(list(ieq.A()))
            A.append(inv(list(ieq.A())))
            b.append(ieq.b())
            b.append(-ieq.b())
    return (np.array(A),np.array(b))


def array_to_H(A,b):
    '''
    Convert two arrays A,b into a polyhedron P
    such that P = {x | Ax >= b}

    :param A: np.array
    :param b: np.array
    :return: Polyhedron()
    '''

    assert(len(A)==len(b))
    return Polyhedron(ieqs=[[b[i]] + list(A[i]) for i in range(len(A))])

In [5]:
def sage_to_pysmt(P):
    '''
    Convert a polyhedron in Sage into its corresponding formula for pysmt

    :param P: Polyhedron
    :return: fNode
    '''

    A,b = H_to_array(P)
    
    if len(A) == 0: # the polyhedron is the entire space
        return TRUE()

    n = len(A[0])
    variables = [Symbol(str(s),REAL) for s in range(n)]

    formula = LE(Real(0),Real(0)) # ???
    for i in range(len(A)):
        la = A[i]
        new_v = []
        for j in range(n):
            v = variables[j]
            new_v.append(Times(v,Real(float(la[j]))))
        formula = And(formula,
                      Not(LT(Plus(new_v),
                             Real(float(-b[i])))))

    return formula

def get_coeff(a,v):
    '''
    Return an array suitable for sage that can be interpreted as an inequality

    :param a: atom of type fNode
    :param variables: list of Symbol
    :return: list of value
    '''
    
    coeffs = [0 for i in range(len(v)+1)]
    if a.is_symbol():
        for i in range(1,len(v)+1):
            if a == v[i-1]:
                coeffs[i] = 1
    elif a.is_plus():
        l_t = []
        for t in a.args():
            l_t.append(get_coeff(t,v))
        for i in range(len(v)+1):
            for l in l_t:
                coeffs[i] +=l[i]
    elif a.is_times():
        l_t = []
        for t in a.args():
            l_t.append(get_coeff(t,v))
        coeffs = l_t.pop()
        while l_t != []:
            u = l_t.pop()
            if coeffs[0] == 0:
                for i in range(1,len(v)+1):
                    if u[i] != 0:
                        coeffs[i] = u[i] * coeffs[0]
                        coeffs[0] = 0
                coeffs[0] *= u[0]
            else:
                for i in range(1,len(v)+1):
                    coeffs[i] = u[i] * coeffs[0]
                    if u[i] != 0:
                        coeffs[0] = 0
                coeffs[0] *= u[0]
    else:
        coeffs[0] = int(a.constant_value())

    return coeffs


def pysmt_to_sage(f):
    '''
    Convert a formula that is a disjunction into its corresponding polyhedron in Sage

    :param f: fNode 
    :return: Polyhedron
    '''
    
    variables = list(f.get_free_variables()) + [Symbol("delta",REAL)]
    n = len(variables)
    ieqs = []
    atoms = [f]
    if f.is_and():
        atoms = f.args()

    for a in atoms:
        left = get_coeff(a.args()[0],variables)
        right = get_coeff(a.args()[1],variables)
        if a.is_le():
            for i in range(len(right)):
                right[i] -= left[i]
            ieqs.append(right)
        if a.is_lt():
            for i in range(len(left)):
                right[i] -= left[i]
                right[-1] = -1
            ieqs.append(right)
            
    return Polyhedron(ieqs=ieqs)


print(pysmt_to_sage(
      And(LE(x,
             Real(0)),
          LT(Real(-5),
             x),
          LE(y,
             Real(5)),
          LT(Real(float(1)),
             y))).Hrepresentation())
print(pysmt_to_sage(And(LE(Times(z,Real(3)),y), GE(Plus(Times(z,Real(5),Real(3)),y,Plus(x,x)),Real(-1)))).Hrepresentation())
print(sage_to_pysmt(Polyhedron(ieqs=[[1,1,0],[2,4,7]])).serialize())

(An inequality (-1, 0, 0) x + 0 >= 0, An inequality (0, -1, 0) x + 5 >= 0, An inequality (0, 1, -1) x - 1 >= 0, An inequality (1, 0, -1) x + 5 >= 0)
(An inequality (0, 1, -3, 0) x + 0 >= 0, An inequality (2, 1, 15, 0) x + 1 >= 0)
(((0.0 <= 0.0) & (! ((('0' * 1.0) + ('1' * 0.0)) < -1.0))) & (! ((('0' * 4.0) + ('1' * 7.0)) < -2.0)))


# DDNF from the formula to Sage

In [6]:
def get_subsolution(formula,variables):
    '''
    Give values for a subset of variables that satisfies the formula

    :param formula: FNode
    :param variables: list of Symbol
    :return: None if the formula is not sat and a list of Real for each variable
    '''

    model = []
    with Solver() as solver:
        solver.add_assertion(formula)
        solver.solve()
        if solver.solve():
            for v in variables:
                model.append(solver.get_value(v))
            return model
        else:
            return None

print(formula.serialize())
get_subsolution(formula,[x]), get_subsolution(formula,[y])

(! ((! (x <= 3.0)) & (! (y < 1.0)) & (y <= 2.0) & ((! (y < 3.0)) | (y < 2.0))))


([4.0], [5/2])

In [7]:
list(formula.args())[0].args(), list(formula.args())[0].is_and(), list(formula.args())[0].is_or()

(((! (x <= 3.0)), (! (y < 1.0)), (y <= 2.0), ((! (y < 3.0)) | (y < 2.0))),
 True,
 False)

In [8]:
def nnf(formula):
    '''
    Compute the negative normal form of a formula

    :param formula: fNode
    :return: fNode
    '''

    if formula.is_and():
        return And([nnf(sf) for sf in formula.args()])
    elif formula.is_or():
        return Or([nnf(sf) for sf in formula.args()])
    elif formula.is_not():
        nf = list(formula.args())[0]
        if nf.is_and():
            return Or([nnf(Not(sf)) for sf in nf.args()])
        elif nf.is_or():
            return And([nnf(Not(sf)) for sf in nf.args()])
        elif nf.is_not():
            return nnf(list(nf.args())[0])
        elif nf.is_le() or nf.is_lt():
            return formula
        elif nf.is_false:
            return TRUE
        elif nf.is_true:
            return FALSE
        else:
            print("Error no operator found", formula)
            assert(False)
    else:
        return formula



print("formula:",formula.serialize())
print("nnf:",nnf(formula).serialize())

formula: (! ((! (x <= 3.0)) & (! (y < 1.0)) & (y <= 2.0) & ((! (y < 3.0)) | (y < 2.0))))
nnf: ((x <= 3.0) | (y < 1.0) | (! (y <= 2.0)) | ((y < 3.0) & (! (y < 2.0))))


In [9]:
def check(formula, variables=[], model=[]):
    '''
    Model-check and sat a formula for a subset of variables

    :param formula: fNode
    :param variables: list of Symbol
    :param model: list of Real
    :return: bool
    '''

    assert(len(variables) == len(model))
    with Solver() as solver:
        solver.add_assertion(formula)
        for i in range(len(variables)):
            solver.add_assertion(Equals(variables[i],model[i]))
        return solver.solve()

def get_true_atoms(formula, variables, p):
    '''
    Compute a subset of atoms that p satisfies in the formula such that this subset is enough to satisfy the formula

    :param formula: fNode
    :param variables: list of Symbol
    :param p: list of Real
    :return: list of fNode
    '''

    if formula.is_and():
        atoms = []
        for sf in formula.args():
            rec = get_true_atoms(sf, variables, p)
            if rec == []:
                return []
            else:
                atoms += rec
        return atoms
    elif formula.is_or():
        for sf in formula.args():
            atoms = get_true_atoms(sf, variables, p)
            if atoms != []:
                return atoms
        return []
    elif formula.is_le() or formula.is_lt():
        if check(formula, variables, p):
            return [formula]
        else:
            return []
    elif formula.is_not():
        nf = formula.args()[0]
        if nf.is_le():
            if check(formula, variables, p):
                left = nf.args()[1]
                right = nf.args()[0]
                return [LT(left,right)]
            else:
                return []
        elif nf.is_lt():
            if check(formula, variables, p):
                left = nf.args()[1]
                right = nf.args()[0]
                return [LE(left,right)]
            else:
                return []
        else:
            print("Error, formula must be in nnf")
            assert(False)
    else:
        print("Error no operator found")
        assert(False)

formula = And(LT(x,Real(0)), Or(Not(LE(x,Real(2))), Not(LE(y,Real(2))), LT(x,Real(2))), Or(LT(y,Real(2)), LE(y,Real(2))))
formula = nnf(formula)
variables = [x,y]
print(formula, variables)
print(get_subsolution(formula,variables))
get_true_atoms(formula, variables, get_subsolution(formula,variables))

((x < 0.0) & ((! (x <= 2.0)) | (! (y <= 2.0)) | (x < 2.0)) & ((y < 2.0) | (y <= 2.0))) [x, y]
[-1.0, 0.0]


[(x < 0.0), (x < 2.0), (y < 2.0)]

In [10]:
def ddnf(formula, variables):
    '''
    Compute a set of disjoint polyedra (when we project over all components exept the last one) equivalent to the formula

    :param formula: fNode
    :param variables: list of Symbol
    :return: list of Polyhedron
    '''

    poly = []
    while check(formula):
        f = nnf(formula)
        atoms = And(get_true_atoms(f, variables, get_subsolution(f,variables)))
        poly.append(pysmt_to_sage(atoms))
        formula = And(formula, Not(atoms))
    return poly



formula = And(Or(And(LE(x,Real(0)), Not(LT(y,Real(1)))),
                 And(LT(x,Real(-2)), LT(y,Real(1)))),
              And(LE(x, Real(0)), Not(LE(x, Real(-5))),LE(y,Real(5)), Not(LE(y, Real(-5)))))
print(formula.serialize())
for P in ddnf(formula,variables):
    print(P.Hrepresentation())
    print()
for P in ddnf(Not(And(LE(x,Real(0)),Not(LE(x,Real(0))))),variables):
    print(P.Hrepresentation())
ddnf(Not(And(LE(x,Real(0)),Not(LE(x,Real(0))))),variables)


((((x <= 0.0) & (! (y < 1.0))) | ((x < -2.0) & (y < 1.0))) & ((x <= 0.0) & (! (x <= -5.0)) & (y <= 5.0) & (! (y <= -5.0))))
(An inequality (-1, 0, -1) x - 2 >= 0, An inequality (-1, 0, 0) x + 0 >= 0, An inequality (0, -1, -1) x + 1 >= 0, An inequality (0, -1, 0) x + 5 >= 0, An inequality (0, 1, -1) x + 5 >= 0, An inequality (1, 0, -1) x + 5 >= 0)

(An inequality (-1, 0, 0) x + 0 >= 0, An inequality (0, -1, 0) x + 5 >= 0, An inequality (1, 0, -1) x + 5 >= 0, An inequality (0, 1, 0) x - 1 >= 0)

(An inequality (-1, 0) x + 0 >= 0,)
(An inequality (1, -1) x + 0 >= 0,)


[A 2-dimensional polyhedron in QQ^2 defined as the convex hull of 1 vertex, 1 ray, 1 line,
 A 2-dimensional polyhedron in QQ^2 defined as the convex hull of 1 vertex, 1 ray, 1 line]

# Quantifier elimination

In [11]:
def minimal_faces(P):
    '''
    Compute a representation of the minimal faces of a polyhedron P
    :param P: Polyhedron
    :return: list of (p=array, l=list of array) with p a point and l a basis of a span
    '''

    return [[p.vector(), [v.vector() for v in P.lines()]] for p in P.vertices()]

P_full_space = Polyhedron(vertices=[[9,41]],lines=[[41,-3],[7,7]])
P_half_space = Polyhedron(ieqs=[[1,1,0]])
P_line = Polyhedron(eqns=[[1,1,1]])
P_half_line = Polyhedron(ieqs=[[1,1,0]], eqns=[[-1,0,1]])
P_polytope = Polyhedron(vertices=[[1,5,3],[4,8,7],[0,0,0],[-5,1,4]])

print("minimal faces:")
print("P_full_space:",minimal_faces(P_full_space))
print("P_half_space:",minimal_faces(P_half_space))
print("P_line:",minimal_faces(P_line))
print("P_half_line:",minimal_faces(P_half_line))
print("P_polytope:",minimal_faces(P_polytope))

minimal faces:
P_full_space: [[(0, 0), [(0, 1), (1, 0)]]]
P_half_space: [[(-1, 0), [(0, 1)]]]
P_line: [[(0, -1), [(1, -1)]]]
P_half_line: [[(-1, 1), []]]
P_polytope: [[(-5, 1, 4), []], [(0, 0, 0), []], [(1, 5, 3), []], [(4, 8, 7), []]]


In [12]:
def min_matrices_faces(P):
    '''
    Compute the list of matrix D associated to each minimal face of the polyhedron F = {(D,E)x = f}

    :param P: Polyhedron
    :param M,c: arrays such that P={Mx <= c}
    :return: list of Polyhedron
    '''

    M,c = H_to_array(P)
    min_faces = minimal_faces(P)

    l_D = []
    for p,_ in min_faces:
        D = []
        for i in range(len(M)):
            l = M[i]
            if np.dot(l,np.array(p)) == -c[i]:
                D.append(l)
        l_D.append(np.array(D))
    return l_D

def list_among(a,b):
    '''
    Compute the list of lists of 'a' indices between 0 and b-1

    :param a: int
    :param b: int
    :return: list of list of int
    '''

    def among(a,b):
        '''
        Compute the list of lists of at most 'a' indices between 0 and b-1

        :param a: int
        :param b: int
        :return: list of list of int
        '''

        if a == 0:
            return [[]]
        elif b == 0:
            return [[]]
        else:
            l1 = list_among(a-1,b-1)
            l2 = list_among(a,b-1)
            return [l+[b-1] for l in l1] + l2

    l = among(a,b)
    res = []
    for t in l:
         if len(t) == a:
             res.append(t)
    return res


def inv_first(M,n):
    '''
    Compute a submatrix of M of maximal rank

    :param M: np.array of np.array
    :param n: int, the rank of M
    :return: np.array of np.array
    '''

    for l in list_among(n,len(M[0])):
        if n == np.linalg.matrix_rank((M.T[l]).T):
            return (M.T[l]).T
    print("Erreur de calcul")
    assert(False)



def get_D(A):
    '''
    Compute the list of matrices representing the minimal faces of a Polyhedron P = {Ax>=b}

    :param A: np.array of np.array
    :return: list of arrays
    '''

    rm = np.linalg.matrix_rank(A)
    assert(rm != 0)
    l_D = []
    for l in list_among(rm,len(A)):
        if rm == np.linalg.matrix_rank(A[l]):
            G = inv_first(A[l],rm)
            Inv = np.linalg.inv(G)
            D = np.zeros((np.shape(A)[1],np.shape(A)[0]))
            for i in range(len(D)):
                i_inv = 0
                if i in l:
                    for j in range(len(Inv[i_inv])):
                        D[i][j] = Inv[i_inv][j]
                else:
                    if i < len(D[i]):
                        D[i][i] = 1
                i_inv = i_inv+1
            l_D.append(D)

    return l_D


def qe_polyhedron(P,vx):
    '''
    Eliminate the variables -which index is not in vx- in the definition of P and returns the equivalent list of polyhedron

    :param P: Polyhedron
    :param vx: list of int
    :return: list of Polyhedron
    '''

    M,c = H_to_array(P)
    A,B = [],[]
    for l in range(len(M[0])):
        if l in vx:
            A.append(M.T[l])
        else:
            B.append(M.T[l])
    A = np.array(A).T
    B = -np.array(B).T


    # Ax >= By +c
    l_D = get_D(A)


    # U_D A(D(By+c)) <= By+c


    return [array_to_H(A.dot(D.dot(B)) - B, c - A.dot(D.dot(c))) for D in l_D]

print("minimal faces matrix:")
print("P_full_space:",min_matrices_faces(P_full_space))
print("P_half_space:",min_matrices_faces(P_half_space))
print("P_line:",min_matrices_faces(P_line))
print("P_half_line:",min_matrices_faces(P_half_line))
print("P_polytope:",min_matrices_faces(P_polytope))
list_among(5,5)
for p in qe_polyhedron(P_polytope,[0,2]):
    print(p.Hrepresentation())

minimal faces matrix:
P_full_space: [array([], dtype=float64)]
P_half_space: [array([[1, 0]])]
P_line: [array([[ 1,  1],
       [-1, -1]])]
P_half_line: [array([[ 0,  1],
       [ 0, -1],
       [ 1,  0]])]
P_polytope: [array([[ 17, -19,  26],
       [-25,  51, -44],
       [ 19, -27,   6]]), array([[ 17, -19,  26],
       [-25,  51, -44],
       [-11,  -5,  12]]), array([[ 17, -19,  26],
       [ 19, -27,   6],
       [-11,  -5,  12]]), array([[-25,  51, -44],
       [ 19, -27,   6],
       [-11,  -5,  12]])]
(An equation (-1022.0) x + 0.0 == 0,)
(An equation (370.8571429) x + 0.0 == 0,)
(An equation (-1291.085714) x + 0.0 == 0,)
(An equation (223.271137) x + 0.0 == 0,)
(An equation (-1407.44898) x + 0.0 == 0,)
(An equation (-234.0) x + 0.0 == 0,)


In [13]:
def fourier_motzkin(P):
    '''
    Compute the Fourier-Motzkin transformation of a Polyhedron on its last component

    :param P: Polyhedron
    :return: Polyhedron
    '''
    
    A,b = H_to_array(P)
    P = []
    for i in range(len(A)):
        P.append([b[i]] + list(A[i]))
    ieqs = []
    deltas_p = []
    deltas_m = []
    for ieq in P:
        if ieq[-1] == 0:
            ieqs.append(ieq[:-1])
        elif ieq[-1] > 0:
            deltas_p.append(ieq)
        else:
            deltas_m.append(ieq)

    for p in deltas_p:
        for m in deltas_m:
            ieq = []
            for i in range(len(p)-1):
                ieq.append(p[-1]*m[i]-m[-1]*p[i])
            ieqs.append(ieq)
    return Polyhedron(ieqs=ieqs)

print(fourier_motzkin(P_full_space).Hrepresentation())
print(fourier_motzkin(P_line).Hrepresentation())
print(fourier_motzkin(P_half_line).Hrepresentation())
print(fourier_motzkin(P_polytope).Hrepresentation())
print()
print([fourier_motzkin(i).Hrepresentation() for i in qe_polyhedron(P_polytope,[0])])

()
()
(An inequality (1) x + 1 >= 0,)
(An inequality (-2, 1) x + 0 >= 0, An inequality (7, -9) x + 44 >= 0, An inequality (1, 5) x + 0 >= 0)

[(An inequality (1568.0) x + 0.0 >= 0, An inequality (-4900.0) x + 29204.0 >= 0), (An inequality (1568.0) x + 0.0 >= 0, An inequality (-4900.0) x + 29204.0 >= 0), (An inequality (1568.0) x + 0.0 >= 0, An inequality (-4900.0) x + 29204.0 >= 0), (An inequality (564.9411764) x + 0.0 >= 0, An inequality (-564.9411765) x + 2824.705882 >= 0)]


# Solving $ \forall \exists \varphi $

In [14]:
def poly_valid(P):
    '''
    Check if the polyhedron is equivalent to TRUE

    :param P: Polyhedron
    :return: None if P is equivalent to TRUE and y a counter-example otherwise 
    '''

    with Solver() as solver:
        solver.add_assertion(Not(sage_to_pysmt(P)))

        if solver.solve():
            return solver.get_model()
        else:
            return None

print("poly valid:")
print("P_full_space:",poly_valid(P_full_space))
print("P_half_space:",poly_valid(P_half_space))
print("P_line:",poly_valid(P_line))
print("P_half_line:",poly_valid(P_half_line))
print("P_polytope:",poly_valid(P_polytope))


poly valid:
P_full_space: None
P_half_space: '0' := -2.0
P_line: '0' := -2.0
'1' := 0.0
P_half_line: '0' := -2.0
'1' := 2.0
P_polytope: '2' := 99/14
'0' := 200/49
'1' := 793/98


In [15]:
variables = [x,y]
formula = And(LT(x,Real(0)),LE(x,y)) #forall y, exists x formula
list_ddnf = ddnf(formula,variables)
print(formula)
print(list_ddnf)


((x < 0.0) & (x <= y))
[A 3-dimensional polyhedron in QQ^3 defined as the convex hull of 1 vertex, 2 rays, 1 line]


In [16]:
def flatten(l):
    '''
    Transform a 2D list into a 1D list

    :param l: list of list
    :return: list
    '''

    res = []
    for u in l:
        res += u
    return res

def solve1(formula,vA):
    '''
    Solve the problem: forall vA, exists vE formula(vA,vE)

    :param formula: fNode
    :param vA: list of Symbol
    :return: boolean
    '''

    variables = formula.get_free_variables()
    vE = []
    for v in variables:
        if v not in vA:
            vE.append(v)

    assert(vA != [])
    assert(vE != [])

    i=0
    iA = []
    iE = []

    for v in variables:
        if v in vA:
            iA.append(i)
        else:
            iE.append(i)
        i = i+1

    # FyEx formula
    list_ddnf = ddnf(formula,vA+vE)
    # Fy U Ex,d P(d,x,y)
    for i in range(len(list_ddnf)):
        list_ddnf[i] = qe_polyhedron(list_ddnf[i],iE)
        for j in range(len(list_ddnf[i])):
            list_ddnf[i][j] = fourier_motzkin(list_ddnf[i][j])
    list_ddnf = flatten(list_ddnf)
    # Fy U P'(y)
    formula = Not(Or([sage_to_pysmt(P) for P in list_ddnf]))
    # not Ey not U P'(y)
    list_ddnf = ddnf(formula,vA)
    # not Ey,d U P''(d,y)
    for i in range(len(list_ddnf)):
        list_ddnf[i] = qe_polyhedron(list_ddnf[i],iA)

        for j in range(len(list_ddnf[i])):
            list_ddnf[i][j] = fourier_motzkin(list_ddnf[i][j])
    list_ddnf = flatten(list_ddnf)
    formula = Or([sage_to_pysmt(P) for P in list_ddnf])

    # not U P'''
    with Solver() as solver:
        solver.add_assertion(Or([sage_to_pysmt(P) for P in list_ddnf]))
        # U P'''
        return not solver.solve() # not U P''' (~ FyEx fomrula)

formula = And(LT(x,Real(0)), LE(x,y))
print(formula.serialize())
solve1(formula,[x])

((x < 0.0) & (x <= y))


False

In [17]:
def solve2(formula,vA):
    '''
    Solve the problem: forall vA, exists vE formula(vA,vE)

    :param formula: fNode
    :param vA: list of Symbol
    :retrun: boolean
    '''

    pass

# Tests

### Generation of formulas

In [18]:
from random import randint

In [19]:
def generate_prop(variables, max_length=10, max_int=10):
    if max_length > 0:
        op = randint(0,3)
        if op == 0:
            return Plus(generate_prop(variables, max_length-1, max_int),generate_prop(variables, max_length-1, max_int))
        elif op == 1:
            return Times(generate_prop(variables,max_length-1,max_int), Real(randint(-max_int,max_int)))
        else:
            atom = randint(0,3)
            if atom == 0:
                return Real(randint(-max_int,max_int))
            else:
                return variables[randint(0,len(variables)-1)]
    else:
        atom = randint(0,3)
        if atom == 0:
            return Real(randint(-max_int,max_int))
        else:
            return variables[randint(0,len(variables)-1)]


def generate_ieq(variables, max_length=10, max_int=10):
    ieq = randint(0,1)
    if ieq == 0:
        return LE(generate_prop(variables, max_length, max_int),generate_prop(variables, max_length, max_int))
    else:
        return LT(generate_prop(variables, max_length, max_int),generate_prop(variables, max_length, max_int))

def generate_phi(variables, max_length):
    if max_length > 0:
        op = randint(0,3)
        if op == 0: # And
            return And(generate_phi(variables, max_length-1), generate_phi(variables, max_length-1))
        elif op == 1: # Or
            return Or(generate_phi(variables, max_length-1), generate_phi(variables, max_length-1))
        elif op == 2: # Not
            return Not(generate_phi(variables, max_length-1))
        else: # Prop
            v = randint(0,len(variables)-1)
            return generate_ieq(variables)
    else:
        v = randint(0,len(variables)-1)
        return generate_ieq(variables)

def generate_formula(n=10, max_length=10):
    vars = [Symbol(str(s),REAL) for s in range(n)]
    vA = vars[:randint(1,len(vars)-1)]
    phi = generate_phi(vars,max_length)
    while(len(list(set(vA).intersection(set(phi.get_free_variables())))) == 0 or 
           len(list(set(phi.get_free_variables()).difference(set(vA)))) == 0):
        vA = vars[:randint(1,len(vars)-1)]
        phi = generate_phi(vars,max_length)
    return phi,list(set(vA).intersection(set(phi.get_free_variables())))

formula = generate_formula()
print(formula[1])
formula[0].serialize()

['0', '1', '2', '3', '4']


"(((! ((('0' <= (('9' * 5.0) + (((-4.0 * 8.0) * 2.0) * -4.0))) & (('1' + 2.0) <= ('2' * -8.0))) & ((! (('2' <= (-8.0 * 8.0)) & ((((((((8.0 * 9.0) * -7.0) + (('1' + '8') * 4.0)) * 0.0) < ((((-6.0 * 5.0) * -10.0) * 9.0) * 10.0)) & ('1' <= '3')) & ((((-3.0 + '5') * -3.0) < 2.0) | ((('9' + ((('4' * -2.0) * -4.0) * -7.0)) + (((('2' + '6') + 10.0) + '2') + 7.0)) < 5.0))) | (-9.0 <= '9')))) | ('6' < '8')))) | (! ((((('3' + '1') <= '3') & ('6' <= ('9' * -7.0))) | (((((((('7' + '3') + ((((-2.0 * -8.0) + '8') * 3.0) + '5')) + ((('8' + ('6' + 2.0)) + 1.0) + ('5' * -5.0))) * -2.0) * 7.0) + ('4' + '8')) + '1') < (((-9.0 * -2.0) * -3.0) * -7.0))) | ((! ((((((((('4' + ('2' + '5')) * 3.0) * -10.0) * 0.0) * -6.0) * 9.0) * 7.0) < ('9' + (((('1' * 8.0) * -10.0) * 10.0) * 5.0))) | ((('1' + '6') <= ('6' + (('0' + '3') + '9'))) | (('1' + '6') <= '0')))) & ((('8' < (('5' * -4.0) + ('0' + ('1' + '7')))) | (! ((1.0 + (('5' + '8') * 5.0)) < ((('9' * -9.0) * 2.0) * -2.0)))) | (! ((('8' + '9') * 2.0) < ('4' * -10

### DDNF

In [20]:
list_formula = [generate_formula(max_length=3) for _ in range(10)]

In [21]:
for f,vA in list_formula:
    print(f.serialize(),vA)
    print(ddnf(f,list(f.get_free_variables())))

(((! (((((((((('4' * 0.0) * 9.0) + (('1' + ('2' + '9')) + '6')) + '3') + (-10.0 + (-6.0 + ((('5' + '3') * 10.0) + 3.0)))) + ('8' + (0.0 + (((('8' * 6.0) * -5.0) + '0') + ('8' * -7.0))))) * -2.0) + ((10.0 + ((8.0 * 4.0) + '7')) * 0.0)) + '8') <= ('6' * -5.0))) & (! (-2.0 <= ('6' + ('2' + (((10.0 + -8.0) * 10.0) * 5.0)))))) | (! (((('2' * 5.0) * -3.0) < ('8' * -7.0)) | ((((('0' * 4.0) + (('2' * -6.0) * 1.0)) * 9.0) + (('5' * -8.0) + (2.0 * 8.0))) <= '0')))) ['5', '6', '7', '0', '2', '1', '3', '4']
[A 11-dimensional polyhedron in QQ^11 defined as the convex hull of 1 vertex, 2 rays, 9 lines, A 6-dimensional polyhedron in QQ^6 defined as the convex hull of 1 vertex, 3 rays, 3 lines, A 11-dimensional polyhedron in QQ^11 defined as the convex hull of 1 vertex, 4 rays, 7 lines]
((-5.0 <= '9') & ((! ('0' <= ('7' + '8'))) | (((('7' * -8.0) + '8') <= '6') | ('9' < ('2' + ('1' * 3.0)))))) ['1', '6', '0', '2']
[A 5-dimensional polyhedron in QQ^5 defined as the convex hull of 1 vertex, 2 rays, 3 li

### Solve1

In [22]:
formula = And(LT(x,Real(0)), LE(x,y))
print(formula.serialize())
print(solve1(formula,[x]))
print(solve1(formula,[y]))

((x < 0.0) & (x <= y))
False
True


In [23]:
formula = Or(LT(x,Real(0)),
             And(Not(LE(x,y)),
                 LT(Plus(Times(x,Real(3)),y,Times(Real(2),z)),Real(5)),
                 LE(z,Real(5)),
                 LE(Real(5),z)))
print(formula.serialize())
print(solve1(formula,[x]))
print(solve1(formula,[y]))

((x < 0.0) | ((! (x <= y)) & (((x * 3.0) + y + (2.0 * z)) < 5.0) & (z <= 5.0) & (5.0 <= z)))
True
True


In [24]:
list_formula = [generate_formula(max_length=3) for _ in range(2)]
for f,vA in list_formula:
    print(f.serialize(),vA)
    print(solve1(f,vA))

(('5' < 5.0) & ((('3' + (6.0 + '0')) + ((3.0 * -1.0) + '9')) <= ((('4' + -3.0) + '6') + '4'))) ['5', '6', '0', '3', '4']
True
(! (((((((('4' + ((-2.0 * -2.0) * -4.0)) * -6.0) * 5.0) + ((((7.0 + '0') * 2.0) * -8.0) * -9.0)) * 6.0) * -2.0) * 7.0) <= '2')) ['0']
True
